### IMPORT LIBRARY

In [ ]:
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File
from office365.runtime.auth.client_credential import ClientCredential
from office365.runtime.client_request_exception import ClientRequestException
import datetime
import pytz
import pandas as pd
import numpy as np
import csv
import glob
import pyodbc
import os 
import json
from io import BytesIO
import io
import platform
from function.PyToSp import *
from tqdm import tqdm
from urllib.parse import quote_plus
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, event
import pyodbc
import requests
import inspect
from validateSamSung import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from urllib.parse import quote_plus
import msal
from itertools import chain
from send_email import *

### CONNECT TO AZURE SQL

In [ ]:
import json
f = open ('database_information.json', "r")
qq = json.loads(f.read())
f.close()
ini_cnt_str ='Driver={driver_str};Server=tcp:hkazdevsqld3vnreserch.database.windows.net,1433;database={database};Uid={username};Pwd={password};Encrypt=yes;Authentication=ActiveDirectoryPassword;Connection Timeout=30;'.format(**qq)
quoted = quote_plus(ini_cnt_str)
cnt_str = 'mssql+pyodbc:///?odbc_connect={}'.format(quoted)
engine = create_engine(cnt_str)

### CONNECT TO SHAREPOINT

In [ ]:
header_BIHub = 'share_point_BIHub'
config_BIHub = read_config_json(config_path, header_BIHub)
BIHub = SharePoint(config_BIHub)
BIHub.check_connect()

In [ ]:
#Tất cả các tỉnh
relative_url = "/sites/BIHub/Shared Documents/Advisory Data/Project/SAMSUNG/Flat file"
sp_object = relative_url.split('/')[2].replace('-','')
list_folder = eval(sp_object).get_content_url(relative_url,return_list_folder=True)

In [ ]:
#lấy năm
list_folder_sub1=[]
for i in list_folder:
    ls=eval(sp_object).get_content_url(i,return_list_folder=True)
    list_folder_sub1=list_folder_sub1+ls

In [105]:
quarter = ['Q2']#Chọn năm cần import
year = ['2023']#Chọn quý cần import
#-------------------------------------------------------
df_summ_file = pd.DataFrame({'Name':[],'ServerRelativeUrl':[], 'TimeLastModified':[], 'ModTime':[], 'Modified_by_ID':[]})
for i in list_folder_sub1:
    if i.split('/')[-2] in year and i.split('/')[-1] in quarter:
        df_summ_file = pd.concat([df_summ_file, eval(sp_object).get_content_url(i)])
list_file = df_summ_file['ServerRelativeUrl'].to_list()
#History file
df_query=pd.DataFrame(df_summ_file).reset_index(drop=True)
df_summ_file = df_summ_file.reset_index(drop=True)

Folder name: 
Files name: /sites/BIHub/Shared Documents/Advisory Data/Project/SAMSUNG/Flat file/2023/Q2/SAMSUNG_HCM_20230630_RetailerInput.xlsx
Files name: /sites/BIHub/Shared Documents/Advisory Data/Project/SAMSUNG/Flat file/2023/Q2/SAMSUNG_HCM_20230630_StreetRetail.xlsx


In [113]:
url = []
for i in list_file:
    data_key = i.split('/')[-1].split('.')[0].split('_')[-1]
    if str(data_key) not in ('x'):
        url.append(i)
url        

['/sites/BIHub/Shared Documents/Advisory Data/Project/SAMSUNG/Flat file/2023/Q2/SAMSUNG_HCM_20230630_RetailerInput.xlsx',
 '/sites/BIHub/Shared Documents/Advisory Data/Project/SAMSUNG/Flat file/2023/Q2/SAMSUNG_HCM_20230630_StreetRetail.xlsx']

## Import

In [116]:
#Create empty df for checking dictionary
df_dict = pd.DataFrame(columns=['File_Name', 'Missing_Values', 'Flag'])
df_temp_flat = pd.DataFrame()
df_flat = pd.DataFrame()
#-------------------------------------------------------
'''Get data''' 

columns_that_need_unidecode = []

for file_url in tqdm(url):
    data, file_name, sector = get_data(relative_url, file_url)
    
    SheetName = file_url.split('/')[-1].split('_')[3].split('.')[0]
    if SheetName == 'RetailerInput':
        table_name = 'SAMSUNG_Retailer_Input'
    else:
        table_name = 'SAMSUNG_Street_Retail'
        columns_that_need_unidecode = ['Contact_Number','Note']

    data = remove_unformated_character(data)
    for i in columns_that_need_unidecode:
        if i not in data.columns:
            data[i] = np.nan
            data[i] = data[i].replace({np.nan: None})
            print(colored('Column {} is added in {}','yellow').format(i, f'{file_name}'))
        else:
            pass   
        data[i] = remove_unicode(data[i])

    
    data = Generate_Additional_Columns(data,df_summ_file,BIHub,engine,file_url)
    data.to_sql(table_name, engine, index=False, if_exists='append', schema='Fresh')

 50%|█████     | 1/2 [00:02<00:02,  2.13s/it]

     Date_Key     Province    Retailer_Group          Retailer_Name  \
0    20230630  Ho Chi Minh  The Gioi Di Dong     The Gioi Di Dong 1   
1    20230630  Ho Chi Minh  The Gioi Di Dong     The Gioi Di Dong 2   
2    20230630  Ho Chi Minh  The Gioi Di Dong     The Gioi Di Dong 3   
3    20230630  Ho Chi Minh  The Gioi Di Dong     The Gioi Di Dong 4   
4    20230630  Ho Chi Minh  The Gioi Di Dong     The Gioi Di Dong 5   
5    20230630  Ho Chi Minh  The Gioi Di Dong     The Gioi Di Dong 6   
6    20230630  Ho Chi Minh  The Gioi Di Dong     The Gioi Di Dong 7   
7    20230630  Ho Chi Minh  The Gioi Di Dong     The Gioi Di Dong 8   
8    20230630  Ho Chi Minh  The Gioi Di Dong     The Gioi Di Dong 9   
9    20230630  Ho Chi Minh  The Gioi Di Dong    The Gioi Di Dong 10   
10   20230630  Ho Chi Minh  The Gioi Di Dong    The Gioi Di Dong 11   
11   20230630  Ho Chi Minh  The Gioi Di Dong    The Gioi Di Dong 12   
12   20230630  Ho Chi Minh  The Gioi Di Dong    The Gioi Di Dong 13   
13   2

100%|██████████| 2/2 [00:07<00:00,  3.58s/it]


In [ ]:
print(f"Rows & Column: {data.shape}")
# Xem thông tin cơ bản của dataframe (số dòng, số cột, kiểu dữ liệu)
print(data.info())

In [ ]:
# Loại bỏ các dòng trùng lặp trong dataframe
data.drop_duplicates(inplace=True)

# Chuyển đổi kiểu dữ liệu của cột Date_Key sang kiểu datetime
data['Date_Key'] = pd.to_datetime(data['Date_Key'])

# Thực hiện thay thế giá trị trong cột "Mobile"
data['Mobile'] = data['Mobile'].replace({0: 'FALSE', 'x': 'TRUE'})

# Thực hiện thay thế giá trị trong cột "Electronics"
data['Electronics'] = data['Electronics'].replace({0: 'FALSE', 'x': 'TRUE'})



In [ ]:
data